In [84]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [85]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [86]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
sc = spark.sparkContext

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
# example inp and out
# s3://comp4442-group-project/data/
# s3://comp4442-group-project/output

import os
import sys


from pyspark import SparkContext, SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import lit

# args = sys.argv
# inp = args[1]
# out = args[2]

sqlContext = SQLContext(sc)

inputList = []
for i in range(2,12):
  index = 0
  if i < 10:
    index = "0" + str(i)
  else:
    index = i

  inputList.append(f"/content/drive/MyDrive/csvdata/detail_record_2017_01_{index}_08_00_00")

# text_file = sc.textFile(inp)
print(inputList)



            

['/content/drive/MyDrive/csvdata/detail_record_2017_01_02_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_03_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_04_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_05_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_06_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_07_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_08_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_09_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_10_08_00_00', '/content/drive/MyDrive/csvdata/detail_record_2017_01_11_08_00_00']


In [130]:
for i ,input in enumerate(inputList):
  text_file = sc.textFile(input)
  # split each line into a list of fields
  # only count the lines that have at least 9 fields, ignore the case that is no special behavior
  counts = text_file.map(lambda line: line.split(",")).filter(lambda line: len(line)>8)


  column_data = counts.map(lambda p: Row(p[0], p[1], p[2], p[3], p[4], \
                                      p[5], p[6], p[7], p[8], p[9] , \
                                      p[10], p[11], p[12], p[13], p[14], \
                                      p[15], p[16], p[17], p[18]))


  column_name = "driverID,carPlateNumber,Latitude,Longitude,Speed,Direction,siteName,Time,isRapidlySpeedup,isRapidlySlowdown,isNeutralSlide,isNeutralSlideFinished,neutralSlideTime,isOverspeed,isOverspeedFinished,overspeedTime,isFatigueDriving,isHthrottleStop,isOilLeak"
  sql = "SELECT first(recordID),first(driverID),first(carPlateNumber),first(Time) \
                              as recordDAY,HOUR(Time) as recordHOUR,\
                              sum(isRapidlySpeedup),sum(isRapidlySlowdown),sum(isNeutralSlide),sum(isNeutralSlideFinished),\
                              sum(neutralSlideTime),sum(isOverspeed),sum(isOverspeedFinished),sum(overspeedTime),sum(isFatigueDriving),\
                              sum(isHthrottleStop),sum(isOilLeak) \
                              FROM summary \
                              GROUP BY driverID,DAY(Time),HOUR(Time)"

  # create schema with the column names
  fields = [StructField(field_name, StringType(), True) for field_name in column_name.split(",")]
  schema = StructType(fields)

  # apply the schema to the RDD
  dataframe = sqlContext.createDataFrame(column_data,schema)
  # add the recordID to store where these record come from
  dataframe = dataframe.withColumn("recordID",lit(input))
  # register the DataFrame as a table.
  dataframe.registerTempTable("summary")

  # execute the SQL query and save the result to the output directory
  group_data = sqlContext.sql(sql)
  group_data.coalesce(1).write.csv("/content/drive/MyDrive/csvdata/data"+str(i))

sc.stop()